# 多线程/多进程的高并发网络爬虫管理算法

In [ ]:
# 多线程版本算法，多线程的优点是：占用内存少，切换简单，CPU利用率高，适用于CPU处理器较好的电脑
import threading

def cheef(start,stop):
    for i in (start,stop):
        url = 'http://localhost/resource/'+i
        print(url.text)

#参数介绍：
#mini_num：执行循环任务的起始值
#max_num：执行循环任务的结束值
#th_num：执行循环任务时，使用的最大线程数(根据自己的电脑性能自行调参)  
def Multi_Threading(mini_num, max_num, th_num):
    difference = max_num - mini_num  # 相差
    remainder = difference % th_num  # 余数
    interval = int((difference - remainder) / th_num)  # 相差的数
    Th_list = []
    for number in range(0, th_num):
        Th_list.append(tuple(((number * interval) + mini_num, (number + 1) * interval + mini_num)))
    if remainder == 0:
        for number in Th_list:
            print('执行范围', number[0], number[1])
            Th = threading.Thread(target=cheef, kwargs={"start": number[0], "stop": number[1]})
            # Th2 = threading.Thread(target=cheef,kwargs={"start":50,"stop":100})
            Th.start()
    else:
        last_tuple = tuple((th_num * interval + mini_num, max_num))
        Th_list.append(last_tuple)
        print(Th_list)
        for number in range(0, len(Th_list) - 1):
            print(Th_list[number])
            Th = threading.Thread(target=cheef,kwargs={"start":Th_list[number][0],"stop":Th_list[number][1]})
            Th.start()
        Th = threading.Thread(target=cheef, kwargs={"start": Th_list[-1][0], "stop": Th_list[-1][1]})
        Th.start()
        
#使用方法：
if __name__ == "__main__":
    Multi_Threading(1,40000,20)

In [ ]:
#多进程版本算法，多进程的优点是：占用内存多，切换复杂，CPU利用率低，适用于内存较高的电脑
import multiprocessing

def cheef(start,stop):
    for i in (start,stop):
        url = 'http://localhost/resource/'+i
        print(url.text)


#参数介绍：
#mini_num：执行循环任务的起始值
#max_num：执行循环任务的结束值
#th_num：执行循环任务时，使用的最大线程数(根据自己的电脑性能自行调参)  
def Multi_Process(mini_num, max_num, th_num):
    difference = max_num - mini_num  # 相差
    remainder = difference % th_num  # 余数
    interval = int((difference - remainder) / th_num)  # 相差的数
    Th_list = []
    for number in range(0, th_num):
        Th_list.append(tuple(((number * interval) + mini_num, (number + 1) * interval + mini_num)))
    if remainder == 0:
        for number in Th_list:
            print('执行范围', number[0], number[1])
            Th = multiprocessing.Process(target=cheef, kwargs={"start": number[0], "stop": number[1]})
            Th.start()
    else:
        last_tuple = tuple((th_num * interval + mini_num, max_num))
        Th_list.append(last_tuple)
        print(Th_list)
        for number in range(0, len(Th_list) - 1):
            print(Th_list[number])
            Th = multiprocessing.Process(target=cheef,kwargs={"start":Th_list[number][0],"stop":Th_list[number][1]})
            Th.start()
        Th = multiprocessing.Process(target=cheef, kwargs={"start": Th_list[-1][0], "stop": Th_list[-1][1]})
        Th.start()
        
#使用方法：
if __name__ == "__main__":
    Multi_Process(1,40000,20)

# 多线程/多进程高并发分布式数据采集算法

In [ ]:
#多线程版高并发分布式数据采集算法
import requests
import threading

def cheef(start,stop):
    for i in (start,stop):
        url = 'http://localhost/resource/'+i
        print(url.text)

class VPN():
    def gain_data():  # 获取代理IP和端口
        try:
            data_dict = {}
            # res = requests.get('https://ip.jiangxianli.com/api/proxy_ips?country=美国')
            res = requests.get('https://niubility666.cn/daili.html')
            res.encoding = 'utf-8'
            null = ''
            data = eval(res.text)
            ip_data = data['data']['data']
            for i in range(0, len(ip_data)):
                ip = ip_data[i]['ip']
                port = ip_data[i]['port']
                data_dict[ip] = port
            # print(data_dict)
            return data_dict
        except Exception as ex:
            print(ex)
            pass

    def connetction():  # 测试接口可用性，并返回可用接口
        try:
            data = VPN.gain_data()  # 获取IP和端口
            available_list = []
            for key in data:
                try:
                    url = "http://pv.sohu.com/cityjson?ie=utf-8"
                    ip = str(key)
                    port = str(data[key])
                    new_data = {
                        "http":'http://' +ip + ":" + port
                    }
                    print("代理后的IP: " + requests.get(url, proxies=new_data).text)
                    available_list.append(new_data)

                except Exception as ex:
                    print('a',ex)
                    pass
            print("切换回自己的IP: " + requests.get(url, proxies={"http": ""}).text)  # 结束后切回自己的IP地址
            return available_list  # 返回可用代理的数组（new_data）
        except Exception as ex:
            print(ex)
            pass

#在这里最大线程数量等于代理IP池的量
def Multi_Threading_Agency(mini_num, max_num):
    proxies_data = VPN.connetction()
    th_num = len(proxies_data)
    difference = max_num - mini_num  # 相差
    remainder = difference % th_num  # 余数
    interval = int((difference - remainder) / th_num)  # 相差的数
    Th_list = []
    for number in range(0, th_num):
        #print(type(proxies_data[number]))
        Th_list.append(tuple(((number * interval) + mini_num, (number + 1) * interval + mini_num,proxies_data[number])))
    if remainder == 0:
        for number in Th_list:
            #print('执行范围', number[0], number[1])
            Th = threading.Thread(target=cheef, kwargs={"start": number[0], "stop": number[1], "agent":number[2]})
            Th.start()
    else:
        last_tuple = tuple((th_num * interval + mini_num, max_num))
        Th_list.append(last_tuple)
        for number in range(0, len(Th_list) - 1):
            Th = threading.Thread(target=cheef, kwargs={"start": Th_list[number][0], "stop": Th_list[number][1],"agent":Th_list[number][2]})
            Th.start()
        Th = threading.Thread(target=cheef, kwargs={"start": Th_list[-1][0], "stop": Th_list[-1][1],"agent":Th_list[number][2]})
        Th.start()

if __name__ == "__main__":
    Multi_Threading_Agency(1,40000,20)

In [ ]:
#多进程版高并发分布式数据采集算法
import requests
import multiprocessing

def cheef(start,stop):
    for i in (start,stop):
        url = 'http://localhost/resource/'+i
        print(url.text)
        
class VPN():
    def gain_data():  # 获取代理IP和端口
        try:
            data_dict = {}
            # res = requests.get('https://ip.jiangxianli.com/api/proxy_ips?country=美国')
            res = requests.get('https://niubility666.cn/daili.html')
            res.encoding = 'utf-8'
            null = ''
            data = eval(res.text)
            ip_data = data['data']['data']
            for i in range(0, len(ip_data)):
                ip = ip_data[i]['ip']
                port = ip_data[i]['port']
                data_dict[ip] = port
            # print(data_dict)
            return data_dict
        except Exception as ex:
            print(ex)
            pass

    def connetction():  # 测试接口可用性，并返回可用接口
        try:
            data = VPN.gain_data()  # 获取IP和端口
            available_list = []
            for key in data:
                try:
                    url = "http://pv.sohu.com/cityjson?ie=utf-8"
                    ip = str(key)
                    port = str(data[key])
                    new_data = {
                        "http":'http://' +ip + ":" + port
                    }
                    print("代理后的IP: " + requests.get(url, proxies=new_data).text)
                    available_list.append(new_data)

                except Exception as ex:
                    print('a',ex)
                    pass
            print("切换回自己的IP: " + requests.get(url, proxies={"http": ""}).text)  # 结束后切回自己的IP地址
            return available_list  # 返回可用代理的数组（new_data）
        except Exception as ex:
            print(ex)
            pass

#在这里最大线程数量等于代理IP池的量
def Multi_Process_Agency(mini_num, max_num):
    proxies_data = VPN.connetction()
    th_num = len(proxies_data)
    difference = max_num - mini_num  # 相差
    remainder = difference % th_num  # 余数
    interval = int((difference - remainder) / th_num)  # 相差的数
    Th_list = []
    for number in range(0, th_num):
        #print(type(proxies_data[number]))
        Th_list.append(tuple(((number * interval) + mini_num, (number + 1) * interval + mini_num,proxies_data[number])))
    if remainder == 0:
        for number in Th_list:
            #print('执行范围', number[0], number[1])
            Th = multiprocessing.Process(target=cheef, kwargs={"start": number[0], "stop": number[1], "agent":number[2]})
            Th.start()
    else:
        last_tuple = tuple((th_num * interval + mini_num, max_num))
        Th_list.append(last_tuple)
        for number in range(0, len(Th_list) - 1):
            Th = multiprocessing.process(target=cheef, kwargs={"start": Th_list[number][0], "stop": Th_list[number][1],"agent":Th_list[number][2]})
            Th.start()
        Th = multiprocessing.process(target=cheef, kwargs={"start": Th_list[-1][0], "stop": Th_list[-1][1],"agent":Th_list[number][2]})
        Th.start()

if __name__ == "__main__":
    Multi_Process_Agency(1,40000,20)